In [12]:
import sqlite3
import pandas as pd
import numpy as np
from reportlab.lib.pagesizes import letter, landscape  # Use landscape orientation
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

# Connect to the SQLite database
conn = sqlite3.connect('.database')

# Query data from the 'Data' table
query = "SELECT * FROM Data"
data = pd.read_sql_query(query, conn)

# Exclude the 'Year' column for statistics calculations
data = data.drop(columns=['year'])

# Calculate the Mean for each column
means = data.mean()

# Calculate the Covariance Matrix
cov_matrix = data.cov()

# Calculate the Variance for each column
variances = data.var()

# Calculate the Correlation Matrix
corr_matrix = data.corr()

# Convert the statistics to formatted tables
mean_table = pd.DataFrame(means, columns=["Mean"]).reset_index()
cov_table = cov_matrix.reset_index()
variance_table = pd.DataFrame(variances, columns=["Variance"]).reset_index()
corr_table = corr_matrix.reset_index()

# Create a PDF document with landscape orientation
pdf_filename = "statistics_report.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=landscape(letter))

# Define styles for tables
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
])

# Create tables
mean_data = [mean_table.columns[:, ].values.tolist()] + mean_table.values.tolist()
cov_data = [cov_table.columns[:, ].values.tolist()] + cov_table.values.tolist()
variance_data = [variance_table.columns[:, ].values.tolist()] + variance_table.values.tolist()
corr_data = [corr_table.columns[:, ].values.tolist()] + corr_table.values.tolist()

mean_table = Table(mean_data)
cov_table = Table(cov_data)
variance_table = Table(variance_data)
corr_table = Table(corr_data)

# Apply styles to tables
mean_table.setStyle(table_style)
cov_table.setStyle(table_style)
variance_table.setStyle(table_style)
corr_table.setStyle(table_style)

# Build the PDF document
pdf_elements = []
pdf_elements.append(mean_table)
pdf_elements.append(cov_table)
pdf_elements.append(variance_table)
pdf_elements.append(corr_table)

doc.build(pdf_elements)

# Close the database connection
conn.close()
